In [2]:
!pip install kaggle

In [3]:
!mkdir .kaggle

In [4]:
import json
token = {"username":"phoebeli9797","key":"de1e943d273136e775a9165770f5b5a1"}
with open('/content/.kaggle/kaggle.json','w') as file:
  json.dump(token,file)

In [13]:
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content

- path is now set to: /content


In [14]:
!kaggle competitions download -c idl-fall2021-hw1p2

  0% 0.00/617k [00:00<?, ?B/s]
100% 617k/617k [00:00<00:00, 91.4MB/s]
 99% 1.91G/1.92G [00:16<00:00, 145MB/s]
100% 1.92G/1.92G [00:16<00:00, 126MB/s]
  0% 0.00/4.03M [00:00<?, ?B/s]
100% 4.03M/4.03M [00:00<00:00, 66.1MB/s]
 97% 239M/246M [00:02<00:00, 95.3MB/s]
100% 246M/246M [00:02<00:00, 114MB/s] 
  0% 0.00/5.16M [00:00<?, ?B/s]
100% 5.16M/5.16M [00:00<00:00, 47.5MB/s]
 94% 227M/241M [00:03<00:00, 63.9MB/s]
100% 241M/241M [00:03<00:00, 67.9MB/s]


In [15]:
!unzip  /content/competitions/idl-fall2021-hw1p2/dev.npy.zip
!unzip  /content/competitions/idl-fall2021-hw1p2/dev_labels.npy.zip
!unzip  /content/competitions/idl-fall2021-hw1p2/sample.csv.zip
!unzip  /content/competitions/idl-fall2021-hw1p2/test.npy.zip
!unzip  /content/competitions/idl-fall2021-hw1p2/train.npy.zip
!unzip  /content/competitions/idl-fall2021-hw1p2/train_labels.npy.zip

Archive:  /content/competitions/idl-fall2021-hw1p2/dev.npy.zip
  inflating: dev.npy                 
Archive:  /content/competitions/idl-fall2021-hw1p2/dev_labels.npy.zip
  inflating: dev_labels.npy          
Archive:  /content/competitions/idl-fall2021-hw1p2/sample.csv.zip
  inflating: sample.csv              
Archive:  /content/competitions/idl-fall2021-hw1p2/test.npy.zip
  inflating: test.npy                
Archive:  /content/competitions/idl-fall2021-hw1p2/train.npy.zip
  inflating: train.npy               
Archive:  /content/competitions/idl-fall2021-hw1p2/train_labels.npy.zip
  inflating: train_labels.npy        


In [16]:
#These libraries help to interact with the operating system and the runtime environment respectively
import os
import sys

#Model/Training related libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
#Dataloader libraries
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

In [17]:
class MLPDataset(Dataset):
  def __init__(self, data,labels, context = 5):
    self.data = np.concatenate(np.load(data,allow_pickle=True),axis=0)
    if labels is not None:
      self.labels = np.concatenate(np.load(labels,allow_pickle=True),axis=0)
    else:
      self.labels = None
    #self.labels = np.concatenate(np.load(labels,allow_pickle=True),axis=0)
    self.length = len(self.data)
    self.context = context
    #pad x first
    self.paddata = np.pad(self.data, ((self.context, self.context),(0,0)), 'constant') # Pad k top, k bottom

  def __len__(self):
    return self.length
  def __getitem__(self,index):
    
    x_out = self.paddata[index:index +2*self.context + 1]
    x_out = torch.tensor(x_out).float().flatten()
    if self.labels is not None:
      return x_out, self.labels[index].astype('int')
    else:
      return x_out

In [18]:
### Load the Data
train_data = 'train.npy'
train_label = 'train_labels.npy'
val_data = 'dev.npy'
val_label = 'dev_labels.npy'
test_data = 'test.npy'

In [19]:
## Dataloaders
# Training dataloader
train_dataset = MLPDataset(train_data, train_label)
train_loader = DataLoader(train_dataset, 
                               shuffle = True, 
                               batch_size = 128, 
                               num_workers=4, 
                               pin_memory = True)

# Validation dataloader
val_dataset = MLPDataset(val_data, val_label)
val_loader = DataLoader(val_dataset, 
                        shuffle = False, 
                        batch_size = 128, 
                        num_workers=4, 
                        pin_memory = True, 
                        drop_last = True)

In [23]:
## Model Architecture definition

class MLP(nn.Module):

    # define model elements
    def __init__(self, size):
        super(MLP, self).__init__()
        
        # Sequential model definition: Input -> Linear -> ReLU -> MaxPool2d -> ReLu -> LeakyRelu -> Relu -> Linear -> Output
        
                                
        self.model = nn.Sequential(nn.Linear(size[0], size[1]), nn.ReLU(),
                                   nn.BatchNorm1d(size[1]),
                                   nn.LeakyReLU(),
                                   nn.Linear(size[1], size[2]))

    def forward(self, x):
        
        # Model forward pass
        
        return self.model(x)

In [24]:
# Model
model = MLP([440,128,71])

# Define Criterion/ Loss function
criterion = nn.CrossEntropyLoss()

# Define Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

print(model)

MLP(
  (model): Sequential(
    (0): Linear(in_features=440, out_features=128, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=128, out_features=71, bias=True)
  )
)


In [25]:
# Train the model

def train_model(train_loader, model):
    training_loss = 0
    
    # Set model in 'Training mode'
    model.train()

    # enumerate mini batches
    # note, i is the batch_idx
    for i, (inputs, targets) in enumerate(train_loader):
        
        # clear the gradients
        optimizer.zero_grad()

        
        # compute the model output
        out = model(inputs)
        
        # calculate loss
        loss = criterion(out, targets)
        
        # Backward pass
        loss.backward()
        
        # Update model weights
        optimizer.step()

        training_loss += loss.item()
    training_loss /= len(train_loader)
    return training_loss

In [26]:
# Evaluate the model

def evaluate_model(val_loader, model):
    
    predictions = []
    actuals = []
    
    # Set model in validation mode
    model.eval()
    total_acc = 0
    for i, (inputs, target) in enumerate(val_loader):
        
        # evaluate the model on the validation set
        out = model(inputs)
                
        # Calculate validation loss
        loss = criterion(inputs, target)
        
        # retrieve numpy array
        #out = out.detach().numpy()
        #actual = target.detach().numpy()

        
        # convert to class labels
        out = torch.argmax(out, 1)
        
        # reshape for stacking
        #actual = actual.flatten()
        #out = out.flatten()
        # store
        #predictions.append(out)
        #actuals.append(actual)
        correct = (out == target).float().sum()
        total_acc += correct
    
    #predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    # Calculate validation accuracy
    #acc = accuracy_score(actuals, predictions)
    return total_acc, loss.item()

In [27]:
# Define number of epochs
epochs = 20

for epoch in range(epochs):
    
    # Train
    training_loss = train_model(train_loader, model)

    # Validation
    val_acc, val_loss = evaluate_model(val_loader, model)

    # scheduler 
    
    # Print log of accuracy and loss
    print("Epoch: "+str(epoch)+", Training loss:" +str(training_loss)+"Validation loss:"+str(val_loss)+
          ", Validation accuracy:"+str(val_acc)+"%")

KeyboardInterrupt: ignored

In [ ]:
test_dataset = MLPDataset(test_data,None)
test_loader = DataLoader(test_dataset, 
                               shuffle = False, 
                               batch_size = 128, 
                               num_workers=4, 
                               pin_memory = True)

In [ ]:
from tqdm.notebook import tqdm

def test_model(model, test_loader):
    predicts = torch.LongTensor()
    
    with torch.no_grad():
        model.eval()

        # no target in test dataset/data loader
        for batch_idx, data in enumerate(tqdm(test_loader)):

            outputs = model(data)

            _, predict = torch.max(outputs.data, 1)
            
            predicts = torch.cat([predicts, predict])
    
    assert predicts.shape[0] == len(test_loader.dataset)

    
    return predicts

In [ ]:
c = test_model(model, test_loader)


  0%|          | 0/14922 [00:00<?, ?it/s]

In [ ]:
print(len(c))

1910012


In [ ]:
id_list = list(range(len(c)))
csv_draft = {'Id':id_list, 'Label':c}

In [ ]:
import pandas as pd
df = pd.DataFrame(csv_draft).to_csv('prediction_3.csv',index = False)


None


In [ ]:
#Submit HW#2
#predict_labels = [item for sublist in ]
#/content/prediction.csv
!kaggle competitions submit /content/prediction.csv -m "submit"

usage: kaggle competitions submit [-h] -f FILE_NAME -m MESSAGE [-q]
                                  [competition]
kaggle competitions submit: error: argument -f/--file is required
